# Definice

In [1]:
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine


In [2]:
pd.set_option('display.max_columns', None)
# Definujte připojení k PostgreSQL
postgres_user = 'postgres'
postgres_password = 'MendeluRulez'
postgres_host = 'localhost'
postgres_port = '5432'
postgres_db = 'mnd_spi_db'

In [3]:
# Vytvořte SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{postgres_user}:{postgres_password}@{postgres_host}:{postgres_port}/{postgres_db}')

In [4]:
data_dir = 'c:/spaces/pythons/mnd_spi_app/data/SLP_VYVOJ/vystup/spi'
#app_dir = 'c:/spaces/pythons/spi_app/projekty/mlv/'
vystup_excel = True
vystup_csv = True

In [5]:
uid = {}
sql_query = "SELECT uid::character varying(255) FROM happ.firm where id_number = 'SLP_VYVOJ'"

# Načtěte data do DataFrame
df = pd.read_sql(sql_query, engine)

#print(df.head())

uid['firm'] = df.at[0, 'uid']
sql_query = f"SELECT uid::character varying(255) FROM happ.lhc where uid_firm = '{uid['firm']}'"
df = pd.read_sql(sql_query, engine)

#print(df.head())

uid['lhc'] = df.at[0, 'uid']

sql_query = f"SELECT uid::character varying(255) FROM happ.lhp where uid_firm = '{uid['firm']}' and uid_lhc = '{uid['lhc']}'"
df = pd.read_sql(sql_query, engine)

#print(df.head())

uid['lhp'] = df.at[0, 'uid']

sql_query = f"SELECT uid::character varying(255) FROM dsuhul.ik where uid_firm = '{uid['firm']}' and uid_lhc = '{uid['lhc']}' and uid_lhp = '{uid['lhp']}'"
df = pd.read_sql(sql_query, engine)

#print(df.head())
uid['ik'] = df.at[0, 'uid']


uid
# firm 738d4938-b36c-4cbe-9b78-b74898c98088
# lhc 41c2b710-8dcd-4d6c-8f41-ee21317f3b4e
# lhp 3a1078aa-a777-429d-93da-1376a3068669
# ik 6d477add-25f0-4c14-9287-dfc3170acd78

{'firm': '738d4938-b36c-4cbe-9b78-b74898c98088',
 'lhc': '41c2b710-8dcd-4d6c-8f41-ee21317f3b4e',
 'lhp': '3a1078aa-a777-429d-93da-1376a3068669',
 'ik': '6d477add-25f0-4c14-9287-dfc3170acd78'}

# Výstupy za LHP

## 10 LHP

In [6]:
sql_query = f"""
 SELECT lhp.lhp_od,
    COALESCE(lhpcz.y_z_akt, 0::numeric) AS y_z_akt,
    COALESCE(lhpcz.t_z_akt, 0::numeric) AS t_z_akt,
    COALESCE(lhpcz.y_z_pred, 0::numeric) AS y_z_pred,
    COALESCE(lhpcz.t_z_pred, 0::numeric) AS t_z_pred,
    COALESCE(lhpcz.y_tez, 0::numeric) AS y_tez,
    COALESCE(lhpcz.t_tez, 0::numeric) AS t_tez,
    COALESCE(lhpcz.y_cil_zasoba, 0::numeric) AS y_cil_zasoba,
    COALESCE(lhpcz.t_cil_zasoba, 0::numeric) AS t_cil_zasoba,
    COALESCE(lhpcz.ft, 0::numeric) AS ft,
    COALESCE(lhpcz.y_cbp, 0::numeric) AS y_cbp,
    COALESCE(lhpcz.t_cbp, 0::numeric) AS t_cbp,
    COALESCE(lhpcz.y_cvt, 0::numeric) AS y_cvt,
    COALESCE(lhpcz.t_cvt, 0::numeric) AS t_cvt,
    COALESCE(lhpcz.y_n, 0::numeric) AS y_n,
    COALESCE(lhpcz.t_n, 0::numeric) AS t_n,
    COALESCE(lhpcz.se_y_z_akt, 0::numeric) AS SE_zas_25,
    COALESCE(lhpcz.se_y_cvt/10, 0::numeric) AS SE_cvt_1,
    COALESCE(round((COALESCE(st_area(lhp.ggeom_ps), 0::double precision) / 10000::double precision)::numeric, 5), 0::numeric) AS plocha_ps,
    COALESCE(lhpcz.par_vypocet, ''::character varying) AS parametry_vypoctu,
    lhpcz.ts_vypocet::character varying(19) AS cas_vypoctu
   FROM happ.lhp lhp,
    happ.lhp_cz01 lhpcz
  WHERE lhp.uid = '{uid['lhp']}'::uuid AND lhpcz.uid = lhp.uid AND lhpcz.dr_zkr::text = ''::text;
"""
df_vysl_010 = pd.read_sql(sql_query, engine)
if vystup_csv:
    df_vysl_010.to_csv(data_dir + '/10_lhp.csv', decimal=',', sep=';', index=False)
if vystup_excel:
    df_vysl_010.to_excel(data_dir + '/10_lhp.xlsx', index=False)
df_vysl_010

,lhp_od,y_z_akt,t_z_akt,y_z_pred,t_z_pred,y_tez,t_tez,y_cil_zasoba,t_cil_zasoba,ft,y_cbp,t_cbp,y_cvt,t_cvt,y_n,t_n,se_zas_25,se_cvt_1,plocha_ps,parametry_vypoctu,cas_vypoctu
0,2024-01-01,235.2,1645871.3,64.6,451997.9,23.2,162384.3,241.6,1690746.5,0.825,9.2,64477.0,74.8,523440.8,784.3,5489137.3,5.1,0.12,7002.10066,#VYPOCET_MINULE;#NEPOCITAT_PLOCHY;,2025-03-20 20:40:23


In [7]:
237491.92/df_vysl_010['t_tez']

0    1.46253
Name: t_tez, dtype: float64

In [8]:
# del df_vysl_010

# Kontrola
df_check = pd.DataFrame({
    'Úroveň': ['LHP'],
    'T Z AKT LHP': df_vysl_010['t_z_akt'].iloc[0],
    'T Z AKT CHK': df_vysl_010['t_z_akt'].iloc[0],
    'Rozdíl Z AKT': df_vysl_010['t_z_akt'].iloc[0] - df_vysl_010['t_z_akt'].iloc[0],
    '% Z AKT': round(100 * (df_vysl_010['t_z_akt'].iloc[0] - df_vysl_010['t_z_akt'].iloc[0])/df_vysl_010['t_z_akt'].iloc[0], 1),
    'T CVT LHP': df_vysl_010['t_cvt'].iloc[0],
    'T CVT CHK': df_vysl_010['t_cvt'].iloc[0],
    'Rozdíl CVT': df_vysl_010['t_cvt'].iloc[0] - df_vysl_010['t_cvt'].iloc[0],
    '% CVT': round(100 * (df_vysl_010['t_cvt'].iloc[0] - df_vysl_010['t_cvt'].iloc[0])/df_vysl_010['t_cvt'].iloc[0], 1),
    'T N LHP': df_vysl_010['t_n'].iloc[0],
    'T N CHK': df_vysl_010['t_n'].iloc[0],
    'Rozdíl N': df_vysl_010['t_n'].iloc[0] - df_vysl_010['t_n'].iloc[0],
    '% N': round(100 * (df_vysl_010['t_n'].iloc[0] - df_vysl_010['t_n'].iloc[0])/df_vysl_010['t_n'].iloc[0], 1)
})
df_check


,Úroveň,T Z AKT LHP,T Z AKT CHK,Rozdíl Z AKT,% Z AKT,T CVT LHP,T CVT CHK,Rozdíl CVT,% CVT,T N LHP,T N CHK,Rozdíl N,% N
0,LHP,1645871.3,1645871.3,0.0,0.0,523440.8,523440.8,0.0,0.0,5489137.3,5489137.3,0.0,0.0


## 20 VS

In [9]:
sql_query = f"""
 SELECT vs.stratum,
    COALESCE(vscz.y_z_akt, 0::numeric) AS y_z_akt,
    COALESCE(vscz.t_z_akt, 0::numeric) AS t_z_akt,
    COALESCE(vscz.y_z_pred, 0::numeric) AS y_z_pred,
    COALESCE(vscz.t_z_pred, 0::numeric) AS t_z_pred,
    COALESCE(vscz.y_tez, 0::numeric) AS y_tez,
    COALESCE(vscz.t_tez, 0::numeric) AS t_tez,
    COALESCE(vscz.y_cil_zasoba, 0::numeric) AS y_cil_zasoba,
    COALESCE(vscz.t_cil_zasoba, 0::numeric) AS t_cil_zasoba,
    COALESCE(lhp.ft, 0::numeric) AS ft,
    COALESCE(vscz.y_cbp, 0::numeric) AS y_cbp,
    COALESCE(vscz.t_cbp, 0::numeric) AS t_cbp,
    COALESCE(vscz.y_cvt, 0::numeric) AS y_cvt,
    COALESCE(vscz.t_cvt, 0::numeric) AS t_cvt,
    COALESCE(vscz.y_n, 0::numeric) AS y_n,
    COALESCE(vscz.t_n, 0::numeric) AS t_n,
    COALESCE(vscz.se_y_z_akt, 0::numeric) AS se_y_z_akt,
    COALESCE(vscz.se_y_cvt, 0::numeric) AS se_y_cvt,
    COALESCE(vscz.se_t_z_akt, 0::numeric) AS se_t_z_akt,
    COALESCE(vscz.se_t_cvt, 0::numeric) AS se_t_cvt,
    round(COALESCE(vsspi.plocha_ps, 0::numeric), 2) AS plocha_ps,
    COALESCE(vscz.y_cbp_vyhl, 0::numeric) AS y_cbp_vyhl,
    COALESCE(vscz.t_cbp_vyhl, 0::numeric) AS t_cbp_vyhl
   FROM dsuhul.vs_spi vsspi,
    dsuhul.vs_spi_dr_cz01 vscz,
    dsuhul.vs vs,
    happ.lhp lhp
  WHERE vsspi.uid_firm = '{uid['firm']}'::uuid AND vsspi.uid_lhc = '{uid['lhc']}'::uuid AND vsspi.uid_lhp = '{uid['lhp']}'::uuid
      AND vscz.uid = vsspi.uid AND vs.uid = vsspi.uid_vs AND vscz.dr_zkr::text = ''::text
      AND lhp.uid = '{uid['lhp']}'::uuid
  ORDER BY vs.stratum;
"""
df_vysl_020 = pd.read_sql(sql_query, engine)
if vystup_csv:
    df_vysl_020.to_csv(data_dir + '/20_vs.csv', decimal=',', sep=';', index=False)
if vystup_excel:
    df_vysl_020.to_excel(data_dir + '/20_vs.xlsx', index=False)
df_vysl_020

,stratum,y_z_akt,t_z_akt,y_z_pred,t_z_pred,y_tez,t_tez,y_cil_zasoba,t_cil_zasoba,ft,y_cbp,t_cbp,y_cvt,t_cvt,y_n,t_n,se_y_z_akt,se_y_cvt,se_t_z_akt,se_t_cvt,plocha_ps,y_cbp_vyhl,t_cbp_vyhl
0,1,209.4,22688.5,198.6,21518.3,35.8,3878.9,188.0,20369.8,0.825,5.2,563.4,46.8,5070.8,752.8,81565.9,18.4,8.4,1993.6,910.1,108.35,8.3,899.3
1,10,202.2,26387.1,226.6,29571.3,68.5,8939.3,250.0,32625.0,0.825,4.9,639.5,32.1,4189.1,827.9,108041.0,36.6,13.6,4776.3,1774.8,130.50,8.8,1148.4
2,11,235.3,72752.4,0.0,0.0,0.0,0.0,250.0,77297.5,0.825,10.8,3339.3,87.1,26930.4,580.7,179546.6,20.0,3.4,6183.8,1051.2,309.19,10.8,3339.3
3,12,201.5,93387.2,0.0,0.0,0.0,0.0,250.0,115865.0,0.825,10.0,4634.6,74.5,34527.8,858.1,397695.0,14.4,2.8,6673.8,1297.7,463.46,10.0,4634.6
4,13,244.6,22760.0,0.0,0.0,0.0,0.0,220.0,20471.0,0.825,8.9,828.1,77.5,7211.4,643.7,59896.3,31.8,5.5,2959.0,511.8,93.05,8.9,828.1
5,14,319.1,40276.8,0.0,0.0,0.0,0.0,250.0,31555.0,0.825,8.1,1022.4,78.1,9857.8,725.9,91623.1,26.8,5.3,3382.7,669.0,126.22,8.1,1022.4
6,2,136.5,7492.5,0.0,0.0,0.0,0.0,224.9,12344.8,0.825,10.2,559.9,69.7,3825.8,775.6,42572.7,22.9,4.4,1257.0,241.5,54.89,10.2,559.9
7,3,139.0,8506.8,213.3,13054.0,107.9,6603.5,188.0,11505.6,0.825,3.8,232.6,23.0,1407.6,664.2,40649.0,18.0,7.5,1101.6,459.0,61.20,8.3,508.0
8,4,215.4,139113.9,271.6,175410.1,100.2,64713.2,206.3,133236.8,0.825,4.9,3164.6,42.1,27189.9,787.7,508728.2,11.2,4.1,7233.4,2647.9,645.84,8.7,5618.8
9,5,202.3,53411.2,0.0,0.0,0.0,0.0,202.7,53516.9,0.825,8.5,2244.2,69.9,18455.0,848.8,224100.2,15.9,2.7,4197.9,712.9,264.02,8.5,2244.2


In [10]:
# kontrola

sums_vs = df_vysl_020[['t_z_akt', 't_cvt', 't_n']].sum()

# Vytvoření nového řádku pro df_check
new_row = pd.DataFrame([{
    'Úroveň': 'VS',
    'T Z AKT LHP': df_vysl_010['t_z_akt'].iloc[0],
    'T Z AKT CHK': sums_vs['t_z_akt'],
    'Rozdíl Z AKT': sums_vs['t_z_akt'] - df_vysl_010['t_z_akt'].iloc[0],
    '% Z AKT': round(100 * (sums_vs['t_z_akt'] - df_vysl_010['t_z_akt'].iloc[0]) / df_vysl_010['t_z_akt'].iloc[0], 1) if df_vysl_010['t_z_akt'].iloc[0] != 0 else 0,
    'T CVT LHP': df_vysl_010['t_cvt'].iloc[0],
    'T CVT CHK': sums_vs['t_cvt'],
    'Rozdíl CVT': sums_vs['t_cvt'] - df_vysl_010['t_cvt'].iloc[0],
    '% CVT': round(100 * (sums_vs['t_cvt'] - df_vysl_010['t_cvt'].iloc[0]) / df_vysl_010['t_cvt'].iloc[0], 1) if df_vysl_010['t_cvt'].iloc[0] != 0 else 0,
    'T N LHP': df_vysl_010['t_n'].iloc[0],
    'T N CHK': sums_vs['t_n'],
    'Rozdíl N': sums_vs['t_n'] - df_vysl_010['t_n'].iloc[0],
    '% N': 100 * (sums_vs['t_n'] - df_vysl_010['t_n'].iloc[0]) / df_vysl_010['t_n'].iloc[0] if df_vysl_010['t_n'].iloc[0] != 0 else 0
}])

# Přidání nového řádku do df_check
df_check = pd.concat([df_check, new_row], ignore_index=True)

df_check

,Úroveň,T Z AKT LHP,T Z AKT CHK,Rozdíl Z AKT,% Z AKT,T CVT LHP,T CVT CHK,Rozdíl CVT,% CVT,T N LHP,T N CHK,Rozdíl N,% N
0,LHP,1645871.3,1645871.3,0.0,0.0,523440.8,523440.8,0.0,0.0,5489137.3,5489137.3,0.0,0.0
1,VS,1645871.3,1645871.3,0.0,0.0,523440.8,523440.8,0.0,0.0,5489137.3,5489137.3,0.0,0.0


In [11]:
del df_vysl_020
del new_row
del sums_vs

## 30 IP

In [12]:
sql_query = f"""
 SELECT strat.stratum,
    il.lokalita,
    ip.plocha,
    ip.odd,
    ip.dil,
    ip.por,
    ip.hos,
	(SELECT pobl.kod
		from
			dsuhul.il_vs_pobl ilp
			, dsuhul.pobl_typy pt
			, dsuhul.pobl pobl
		where
			ilp.uid_il = il.uid
			and ilp.uid_vs = strat.uid
			and pobl.uid = ilp.uid_pobl
			and pt.uid = pobl.uid_tpobl
			and pt.kod = 'LS_KOD'
	) ls_kod,
	(SELECT pobl.kod
		from
			dsuhul.il_vs_pobl ilp
			, dsuhul.pobl_typy pt
			, dsuhul.pobl pobl
		where
			ilp.uid_il = il.uid
			and ilp.uid_vs = strat.uid
			and pobl.uid = ilp.uid_pobl
			and pt.uid = pobl.uid_tpobl
			and pt.kod = 'TVL'
	) tvl,
	(SELECT pobl.kod
		from
			dsuhul.il_vs_pobl ilp
			, dsuhul.pobl_typy pt
			, dsuhul.pobl pobl
		where
			ilp.uid_il = il.uid
			and ilp.uid_vs = strat.uid
			and pobl.uid = ilp.uid_pobl
			and pt.uid = pobl.uid_tpobl
			and pt.kod = 'ZONA'
	) zona,
    ip.nadm_vyska,
    ip.ek,
    COALESCE(er.er, ''::bpchar) AS er,
    ip.poz_les,
        CASE
            WHEN ip.poz_les::text = '100'::text THEN ip.status
            ELSE ''::character varying
        END AS status,
    COALESCE(ipcz.y_z_akt, 0::numeric) AS y_z_akt,
    COALESCE(ipcz.y_z_pred, 0::numeric) AS y_z_pred,
    COALESCE(ipcz.y_tez, 0::numeric) AS y_tez,
    COALESCE(ip.cil_zasoba, 0::numeric) AS cil_zasoba,
    COALESCE(ipcz.perioda, 0::numeric) AS perioda,
    COALESCE(ip.vyr_doba, 0::numeric) AS vyr_doba,
    COALESCE(ipcz.y_cbp, 0::numeric) AS y_cbp,
    COALESCE(ipcz.y_cvt, 0::numeric) AS y_cvt,
    COALESCE(ipcz.y_n, 0::numeric) AS y_n,
    COALESCE(ipcz.vyr_doba_vypocet, 0::numeric) AS vyr_doba_vypocet,
    COALESCE(ipcz.perioda, 0::numeric) AS perioda_vypocet,
    COALESCE(ipcz.ft_vypocet, 0::numeric) AS ft_vypocet,
    COALESCE(ipcz.cbp_vypocet, 0::numeric) AS cbp_vypocet,
    COALESCE(ipcz.cil_zasoba_vypocet, 0::numeric) AS cil_zasoba_vypocet,
    ip.cbp_max AS cbp_vyhl,
    ip.cil_zasoba_min AS min_cil_zasoba_vyhl,
    COALESCE(ip.pozn_plocha, ''::character varying) AS pozn_plocha
   FROM dsuhul.ip ip
     LEFT JOIN dsuhul.ip_cz01 ipcz ON ipcz.uid = ip.uid AND ipcz.dr_zkr::text = ''::text
     LEFT JOIN dsuhul.l_er_cz01 er ON er.ek = ip.ek,
    dsuhul.il il,
    dsuhul.vs_spi vs,
    dsuhul.vs_spi_dr_cz01 vscz,
    dsuhul.vs strat
  WHERE ip.uid_firm = '{uid['firm']}'::uuid AND ip.uid_lhc = '{uid['lhc']}'::uuid AND ip.uid_lhp = '{uid['lhp']}'::uuid
      AND vs.uid_firm = ip.uid_firm AND vs.uid_lhc = ip.uid_lhc AND vs.uid_lhp = ip.uid_lhp AND vs.uid_ik = ip.uid_ik AND vs.uid_vs = ip.uid_vs AND vscz.uid = vs.uid AND vscz.dr_zkr::text = ''::text 
      AND il.uid = ip.uid_il AND strat.uid = vs.uid_vs
  ORDER BY (il.lokalita::integer)
"""
df_vysl_030 = pd.read_sql(sql_query, engine)
if vystup_csv:
    df_vysl_030.to_csv(data_dir + '/30_ip.csv', decimal=',', sep=';', index=False)
if vystup_excel:
    df_vysl_030.to_excel(data_dir + '/30_ip.xlsx', index=False)
df_vysl_030

,stratum,lokalita,plocha,odd,dil,por,hos,ls_kod,tvl,zona,nadm_vyska,ek,er,poz_les,status,y_z_akt,y_z_pred,y_tez,cil_zasoba,perioda,vyr_doba,y_cbp,y_cvt,y_n,vyr_doba_vypocet,perioda_vypocet,ft_vypocet,cbp_vypocet,cil_zasoba_vypocet,cbp_vyhl,min_cil_zasoba_vyhl,pozn_plocha
0,1,10000001.0,1.0,79.0,C,a,9,None,None,None,331.60,S,B,100,100,85.801,138.911,100.20,150.0,8.9,50.0,5.3,26.862,848.4,50.0,8.9,0.825,5.3,188.0,8.3,188.0,
1,1,10000002.0,1.0,79.0,B,a,9,None,None,None,309.95,S,B,100,100,108.287,178.666,100.60,150.0,8.9,50.0,3.4,14.897,210.0,50.0,8.9,0.825,3.4,188.0,8.3,188.0,
2,1,10000003.0,1.0,79.0,B,a,9,None,None,None,282.68,C,B,100,100,206.329,151.498,22.48,150.0,8.9,50.0,8.7,74.799,930.0,50.0,8.9,0.825,8.7,188.0,8.3,188.0,
3,1,10000012.0,1.0,82.0,B,a,9,None,None,None,351.38,B,B,100,100,111.651,71.769,0.00,150.0,8.9,50.0,4.5,24.527,910.0,50.0,8.9,0.825,4.5,188.0,8.3,188.0,
4,1,10000013.0,1.0,82.0,B,a,9,None,None,None,338.23,B,B,100,100,315.311,248.534,0.00,150.0,8.9,50.0,7.5,82.881,1278.4,50.0,8.9,0.825,7.5,188.0,8.3,188.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,14,140584850.0,1.0,120.0,A,a,11,None,None,None,315.10,J,J,100,200,345.117,0.000,0.00,250.0,0.0,50.0,8.3,84.169,530.0,50.0,0.0,0.825,8.3,250.0,8.3,188.0,
1206,14,140625052.0,1.0,102.0,C,a,11,None,None,None,446.16,J,J,100,200,114.806,0.000,0.00,250.0,0.0,50.0,9.7,57.718,190.0,50.0,0.0,0.825,9.7,250.0,9.7,242.0,
1207,14,140629081.0,1.0,101.0,A,a,11,None,None,None,437.46,X,E,100,200,315.786,0.000,0.00,250.0,0.0,50.0,5.0,52.105,2199.4,50.0,0.0,0.825,5.0,250.0,5.0,158.0,
1208,14,140632058.0,1.0,102.0,C,a,11,None,None,None,395.61,J,J,100,200,136.576,0.000,0.00,250.0,0.0,50.0,8.3,49.760,83.5,50.0,0.0,0.825,8.3,250.0,8.3,188.0,


In [13]:
del df_vysl_030

In [14]:
sql_query = f"""
select vs.stratum, il.lokalita, ipcz.y_z_akt ip_y_z_akt, ipcz.y_cvt ip_y_cvt
	, vsc.y_z_akt vs_y_z_akt, vsc.y_cvt vs_y_cvt
	, (
		select count(*)
		from
			dsuhul.vs_spi vs_spi_mx,
			dsuhul.il_vs_spi ivs_mx
			, dsuhul.il il_mx
		where
			vs_spi_mx.uid = vs_spi.uid
			and ivs_mx.uid_vs_spi = vs_spi_mx.uid
			and il_mx.uid = ivs_mx.uid_il
	) mx
	, (
		select count(*)
		from
			dsuhul.vs_spi vs_spi_m2,
			dsuhul.il_vs_spi ivs_m2
			, dsuhul.il il_m2
			, dsuhul.ip ip_m2
		where
			vs_spi_m2.uid = vs_spi.uid
			and ivs_m2.uid_vs_spi = vs_spi_m2.uid
			and il_m2.uid = ivs_m2.uid_il
			and ip_m2.uid_il = il_m2.uid and ip_m2.prist = '100' and ip_m2.poz_les = '100' and ip_m2.kat_poz = '100'
	) m2
-- sum(vs_spi.m_nom), sum(vs_spi.plocha_ps)
FROM
	dsuhul.ip ip
	, dsuhul.ip_cz01 ipcz
	, dsuhul.il il
	, dsuhul.il_vs_spi ivs
	, dsuhul.vs_spi vs_spi
	, dsuhul.vs vs
	, dsuhul.vs_spi_dr_cz01 vsc
WHERE
	ip.uid_firm = '756b86e6-f325-46a3-bb57-46f091890a14'
	and ipcz.uid = ip.uid
	and il.uid = ip.uid_il
	and ipcz.dr_zkr = '' and ipcz.tlt = 0
	and ivs.uid_il = il.uid
	and vs_spi.uid = ivs.uid_vs_spi
	and vs.uid = vs_spi.uid_vs
	and vsc.uid = vs_spi.uid and vsc.dr_zkr = '' and vsc.tlt = 0
order by vs.stratum, il.lokalita::integer
"""
df_vysl_030_var = pd.read_sql(sql_query, engine)
if vystup_csv:
    df_vysl_030_var.to_csv(data_dir + '/30_ip_var.csv', decimal=',', sep=';', index=False)
if vystup_excel:
    df_vysl_030_var.to_excel(data_dir + '/30_ip_var.xlsx', index=False)
df_vysl_030_var

,stratum,lokalita,ip_y_z_akt,ip_y_cvt,vs_y_z_akt,vs_y_cvt,mx,m2


In [15]:
df_vysl_030_var

,stratum,lokalita,ip_y_z_akt,ip_y_cvt,vs_y_z_akt,vs_y_cvt,mx,m2


In [16]:
del df_vysl_030_var

## 40 VZ

In [17]:
sql_query = f"""
  SELECT vs.stratum,
    il.lokalita::integer,
    ip.plocha::integer,
    ip.hos,
	(SELECT pobl.kod
		from
			dsuhul.il_vs_pobl ilp
			, dsuhul.pobl_typy pt
			, dsuhul.pobl pobl
		where
			ilp.uid_il = il.uid
			and ilp.uid_vs = vs.uid
			and pobl.uid = ilp.uid_pobl
			and pt.uid = pobl.uid_tpobl
			and pt.kod = 'LS_KOD'
	) ls_kod,
	(SELECT pobl.kod
		from
			dsuhul.il_vs_pobl ilp
			, dsuhul.pobl_typy pt
			, dsuhul.pobl pobl
		where
			ilp.uid_il = il.uid
			and ilp.uid_vs = vs.uid
			and pobl.uid = ilp.uid_pobl
			and pt.uid = pobl.uid_tpobl
			and pt.kod = 'TVL'
	) tvl,
	(SELECT pobl.kod
		from
			dsuhul.il_vs_pobl ilp
			, dsuhul.pobl_typy pt
			, dsuhul.pobl pobl
		where
			ilp.uid_il = il.uid
			and ilp.uid_vs = vs.uid
			and pobl.uid = ilp.uid_pobl
			and pt.uid = pobl.uid_tpobl
			and pt.kod = 'ZONA'
	) zona,
    COALESCE(vz.podplocha, 0::numeric)::integer AS podplocha,
    vz.kmen::integer,
    COALESCE(vz.dr_zkr, ''::character varying) AS dr_zkr,
        CASE
            WHEN COALESCE(vz.parez, ''::character varying)::text = ANY (ARRAY['200'::character varying::text, '300'::character varying::text]) THEN 'Starý pařez'::text
            WHEN COALESCE(vz.parez, ''::character varying)::text = '400'::text THEN 'Pařez z těžby'::text
            WHEN COALESCE(vz.parez, ''::character varying)::text = '500'::text THEN 'Pařez zlomu'::text
            WHEN COALESCE(vz.parez, ''::character varying)::text = '400'::text THEN 'Pařez z těžby souše'::text
            WHEN COALESCE(vz.parez, ''::character varying)::text = '500'::text THEN 'Pařez zlomu souše'::text
            WHEN COALESCE(vz.zlom_vyvrat, ''::character varying)::text = '200'::text THEN 'Ohyb'::text
            WHEN COALESCE(vz.zlom_vyvrat, ''::character varying)::text = '300'::text THEN 'Vrškový zlom'::text
            WHEN COALESCE(vz.zlom_vyvrat, ''::character varying)::text = '400'::text THEN 'Korunový zlom'::text
            WHEN COALESCE(vz.zlom_vyvrat, ''::character varying)::text = '500'::text THEN 'Kmenový zlom'::text
            WHEN COALESCE(vz.zlom_vyvrat, ''::character varying)::text = '600'::text THEN 'Náhradní vrchol'::text
            WHEN COALESCE(vz.zlom_vyvrat, ''::character varying)::text = '700'::text THEN 'Živý vývrat'::text
            WHEN COALESCE(vz.dvojak, ''::character varying)::text = '200'::text THEN 'Dvoják'::text
            ELSE 'Živý kmen'::text
        END AS typ_vzorniku,
    vz.azimut_km,
    vz.vzd_km,
    COALESCE(vz.opak_ident_km, ''::character varying) AS opak_ident_km,
    COALESCE(vz.parez, ''::character varying) AS parez,
    COALESCE(vz.pol_parez, ''::character varying) AS pol_parez,
    COALESCE(vz.vyklizeni_km, ''::character varying) AS vyklizeni_km,
    COALESCE(vz.sous, ''::character varying) AS sous,
    COALESCE(vz.zlom_vyvrat, ''::character varying) AS zlom_vyvrat,
    COALESCE(vz.dvojak, ''::character varying) AS dvojak,
    COALESCE(vz.kmen_pred, 0::numeric)::integer AS kmen_pred,
    COALESCE(vz.parez_pred, ''::character varying) AS parez_pred,
    COALESCE(vz.sous_pred, ''::character varying) AS sous_pred,
    COALESCE(vz.zlom_vyvrat_pred, ''::character varying) AS zlom_vyvrat_pred,
    COALESCE(vz.komp_zmeny, ''::character varying) AS komp_zmeny,
    COALESCE(vz.tloustka_km, 0::numeric) AS tloustka_km,
    COALESCE(vz.vyska, 0::numeric) AS mvyska,
    COALESCE(vz.mod_vys, 0::numeric) AS mod_vys,
    COALESCE(vz.tloustka_km_pred, 0::numeric) AS tloustka_km_pred,
    COALESCE(vz.vyska_pred, 0::numeric) AS mvyska_pred,
    COALESCE(vz.mod_vys_pred, 0::numeric) AS mod_vys_pred,
    COALESCE(vz.pno, 0::numeric) AS pno,
    COALESCE(vz.to_mv_bk, 0::numeric) AS to_mv_bk,
    COALESCE(vz.to_modv_bk, 0::numeric) AS to_modv_bk,
    COALESCE(vz.pno_pred, 0::numeric) AS pno_pred,
    COALESCE(vz.to_mv_bk_pred, 0::numeric) AS to_mv_bk_pred,
    COALESCE(vz.to_modv_bk_pred, 0::numeric) AS to_modv_bk_pred,
        CASE
            WHEN COALESCE(vz.vzornik2, false) THEN 'A'::text
            ELSE 'N'::text
        END AS vzornik2,
        CASE
            WHEN COALESCE(vz.vzornik2_pred, false) THEN 'A'::text
            ELSE 'N'::text
        END AS vzornik2_pred,
    COALESCE(vz.zz1, 0::numeric) AS zz1,
    COALESCE(vz.zz2, 0::numeric) AS zz2,
    COALESCE(vz.d13_depl, 0::numeric) AS d13_depl,
    COALESCE(vz.mod_vys_depl, 0::numeric) AS mod_vys_depl,
    COALESCE(vz.to_modv_bk_depl, 0::numeric) AS to_modv_bk_depl,
    COALESCE(vz.to_modv_sk_depl, 0::numeric) AS to_modv_sk_depl,
    COALESCE(vz.zz1_depl, 0::numeric) AS zz1_depl,
    COALESCE(vz.zz2_depl, 0::numeric) AS zz2_depl,
    coalesce(vzc.y_z_akt_1, 0::numeric) AS y_z_akt_1,
    coalesce(vzc.y_z_akt_2, 0::numeric) AS y_z_akt_2,
    coalesce(vzc.y_z_akt, 0::numeric) AS y_z_akt,
    coalesce(vzc.y_z_pred_1, 0::numeric) AS y_z_pred_1,
    coalesce(vzc.y_z_pred_2, 0::numeric) AS y_z_pred_2,
    coalesce(vzc.y_z_pred, 0::numeric) AS y_z_pred,
    coalesce(vzc.y_tez, 0::numeric) AS y_tez,
    coalesce(vzc.y_z_akt_1, 0::numeric) - coalesce(vzc.y_z_pred_1, 0::numeric)
    + coalesce(vzc.y_z_akt_2, 0::numeric) - coalesce(vzc.y_z_pred_2, 0::numeric)
    + coalesce(vzc.y_tez, 0::numeric) as prirust,
    COALESCE(vz.pozn_km, ''::character varying) AS pozn_km
   FROM dsuhul.ip ip,
    dsuhul.il il,
    dsuhul.vz vz
        left outer join dsuhul.vz_cz01 vzc on vz.uid = vzc.uid,
    dsuhul.vs vs
  WHERE ip.uid_firm = '{uid['firm']}'::uuid AND ip.uid_lhc = '{uid['lhc']}'::uuid AND ip.uid_lhp = '{uid['lhp']}'::uuid
      AND il.uid = ip.uid_il AND vz.uid_ip = ip.uid AND vs.uid = ip.uid_vs
      AND vz.zapocitat
  ORDER BY il.lokalita;
"""
df_vysl_040 = pd.read_sql(sql_query, engine)
if vystup_csv:
    df_vysl_040.to_csv(data_dir + '/40_vz.csv', decimal=',', sep=';', index=False)
if vystup_excel:
    df_vysl_040.to_excel(data_dir + '/40_vz.xlsx', index=False)
df_vysl_040

,stratum,lokalita,plocha,hos,ls_kod,tvl,zona,podplocha,kmen,dr_zkr,typ_vzorniku,azimut_km,vzd_km,opak_ident_km,parez,pol_parez,vyklizeni_km,sous,zlom_vyvrat,dvojak,kmen_pred,parez_pred,sous_pred,zlom_vyvrat_pred,komp_zmeny,tloustka_km,mvyska,mod_vys,tloustka_km_pred,mvyska_pred,mod_vys_pred,pno,to_mv_bk,to_modv_bk,pno_pred,to_mv_bk_pred,to_modv_bk_pred,vzornik2,vzornik2_pred,zz1,zz2,d13_depl,mod_vys_depl,to_modv_bk_depl,to_modv_sk_depl,zz1_depl,zz2_depl,y_z_akt_1,y_z_akt_2,y_z_akt,y_z_pred_1,y_z_pred_2,y_z_pred,y_tez,prirust,pozn_km
0,1,10000001,1,9,None,None,None,2,8,LP,Pařez z těžby,67.4,5.10,,400,100,100,100,,100,8,100,,,400,15.3,0.0,14.9,14.7,15.8,14.9,0.0,0.000,0.122,0.0,0.000,0.112,N,N,0.01539,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,7.277,0.000,7.277,7.28,0.003,
1,1,10000001,1,9,None,None,None,2,7,DBZ,Pařez z těžby,51.4,5.87,,400,100,100,300,100,100,7,100,,,400,12.2,0.0,12.5,12.1,0.0,12.9,0.0,0.000,0.047,0.0,0.000,0.048,N,N,0.01539,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,3.119,0.000,3.119,3.12,0.001,
2,1,10000001,1,9,None,None,None,2,10,DBZ,Živý kmen,95.0,2.64,,100,100,,100,,100,10,100,,,100,16.8,12.9,15.7,14.3,11.0,15.9,0.0,0.102,0.127,0.0,0.000,0.091,A,N,0.01539,0.00246,0.0,0.0,0.0,0.0,0.0,0.0,8.252,-10.163,-1.911,5.913,0.000,5.913,0.00,-7.824,
3,1,10000001,1,9,None,None,None,2,9,BO,Pařez z těžby,83.5,5.84,,400,100,100,100,100,100,9,100,,,400,16.5,0.0,15.3,16.0,0.0,15.1,0.0,0.000,0.127,0.0,0.000,0.117,N,N,0.01539,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,7.602,0.000,7.602,7.60,-0.002,
4,1,10000001,1,9,None,None,None,2,15,HB,Živý kmen,134.3,0.62,,100,100,,100,,100,15,100,,,100,13.1,11.6,13.3,11.0,10.5,13.3,0.0,0.056,0.067,0.0,0.032,0.044,A,A,0.01539,0.00149,0.0,0.0,0.0,0.0,0.0,0.0,4.353,-7.383,-3.030,15.548,-11.429,4.119,0.00,-7.149,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17084,14,140633918,1,11,None,None,None,1,7,AK,Živý kmen,158.4,1.60,,100,,,100,100,100,0,,,,,9.7,8.6,10.6,0.0,0.0,0.0,0.0,0.015,0.018,0.0,0.000,0.000,A,N,0.00209,0.00075,0.0,0.0,0.0,0.0,0.0,0.0,8.612,-4.000,4.612,0.000,0.000,0.000,0.00,4.612,
17085,14,140633918,1,11,None,None,None,2,1,JV,Živý kmen,317.5,6.21,,100,,,100,100,100,0,,,,,13.0,0.0,13.2,0.0,0.0,0.0,0.0,0.000,0.077,0.0,0.000,0.000,N,N,0.00677,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,11.374,0.000,11.374,0.000,0.000,0.000,0.00,11.374,
17086,14,140633918,1,11,None,None,None,3,10,BO,Živý kmen,4.4,6.39,,100,,,100,100,100,0,,,,,47.0,15.6,30.7,0.0,0.0,0.0,0.0,1.103,2.140,0.0,0.000,0.000,A,N,0.02316,0.00814,0.0,0.0,0.0,0.0,0.0,0.0,92.401,-127.396,-34.995,0.000,0.000,0.000,0.00,-34.995,h
17087,14,140633918,1,11,None,None,None,1,6,AK,Živý kmen,128.1,1.59,,100,,,300,100,100,0,,,,,8.0,0.0,9.1,0.0,0.0,0.0,0.0,0.000,0.014,0.0,0.000,0.000,N,N,0.00166,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,8.434,0.000,8.434,0.000,0.000,0.000,0.00,8.434,


In [18]:
del df_vysl_040

## 50 HOS

In [19]:
sql_query = f"""
 SELECT p.kod, p.nazev,
    round(pc.y_n, 1) AS y_n,
    round(pc.t_n, 0) AS pocet_kh,
    pc.y_z_akt AS y_z_akt,
    pc.t_z_akt AS zasoba,
    pc.y_cbp,
    pc.t_cbp,
    pc.y_cvt,
    pc.t_cvt,
	  round(p.plocha_ps, 2) plocha_ps
   FROM dsuhul.pobl_typy pt,
    dsuhul.pobl p,
    dsuhul.pobl_cz01 pc
  WHERE pt.uid_lhp = '{uid['lhp']}'::uuid AND pt.kod::text = 'HOS'::text AND p.uid_tpobl = pt.uid AND pc.uid_pobl = p.uid AND pc.dr_zkr::text = ''::text
  ORDER BY p.kod
"""
df_vysl_050_hos = pd.read_sql(sql_query, engine)
if vystup_csv:
    df_vysl_050_hos.to_csv(data_dir + '/50_hos_sumar.csv', decimal=',', sep=';', index=False)
if vystup_excel:
    df_vysl_050_hos.to_excel(data_dir + '/50_hos_sumar.xlsx', index=False)
df_vysl_050_hos

,kod,nazev,y_n,pocet_kh,y_z_akt,zasoba,y_cbp,t_cbp,y_cvt,t_cvt,plocha_ps
0,01,Účelové hospodářství v modelu Dauerwald nižšíc...,803.4,742966.0,212.4,196413.0,6.0,5575.4,50.8,46972.5,924.94
1,02,Účelové hospodářství v modelu Dauerwald středn...,802.4,3368883.0,240.7,1010801.1,10.0,41895.3,80.8,339241.7,4200.27
2,03,Účelové hospodářství v modelu Dauerwald středn...,783.7,335528.0,205.5,87981.1,8.2,3501.8,60.1,25744.8,428.19
3,04,Účelové hospodářství v modelu výběrný les,692.3,97037.0,340.7,47749.6,9.7,1363.1,95.8,13427.9,140.16
4,05,Účelové hospodářství v modelu výběrný les v ge...,652.1,93809.0,374.6,53886.4,8.9,1286.8,97.8,14065.8,113.59
5,06,Účelové hospodářství v modelu mozaikový les,579.4,177316.0,236.1,72247.1,10.9,3324.6,87.3,26725.2,306.13
6,07,Účelové hospodářství v modelu přírůstném,858.1,397536.0,201.5,93335.5,10.0,4632.5,74.5,34508.0,463.32
7,08,Účelové hospodářství v modelu nízký les,664.2,40648.0,139.0,8505.8,3.8,230.9,23.0,1410.2,61.52
8,09,Účelové hospodářství v modelu střední les,770.2,125463.0,185.6,30225.1,6.9,1123.0,54.5,8875.2,163.20
9,10,Účelové hospodářství v modelu extenzivní les,644.6,61891.0,242.7,23302.5,8.9,856.5,77.2,7411.7,96.43


In [20]:
# kontrola
sumar = df_vysl_050_hos[['zasoba', 't_cvt', 'pocet_kh']].sum()

# Vytvoření nového řádku pro df_check
new_row = pd.DataFrame([{
    'Úroveň': 'HOS',
    'T Z AKT LHP': df_vysl_010['t_z_akt'].iloc[0],
    'T Z AKT CHK': sumar['zasoba'],
    'Rozdíl Z AKT': sumar['zasoba'] - df_vysl_010['t_z_akt'].iloc[0],
    '% Z AKT': round(100 * (sumar['zasoba'] - df_vysl_010['t_z_akt'].iloc[0]) / df_vysl_010['t_z_akt'].iloc[0], 1) if df_vysl_010['t_z_akt'].iloc[0] != 0 else 0,
    'T CVT LHP': df_vysl_010['t_cvt'].iloc[0],
    'T CVT CHK': sumar['t_cvt'],
    'Rozdíl CVT': sumar['t_cvt'] - df_vysl_010['t_cvt'].iloc[0],
    '% CVT': round(100 * (sumar['t_cvt'] - df_vysl_010['t_cvt'].iloc[0]) / df_vysl_010['t_cvt'].iloc[0], 1) if df_vysl_010['t_cvt'].iloc[0] != 0 else 0,
    'T N LHP': df_vysl_010['t_n'].iloc[0],
    'T N CHK': sumar['pocet_kh'],
    'Rozdíl N': sumar['pocet_kh'] - df_vysl_010['t_n'].iloc[0],
    '% N': round(100 * (sumar['pocet_kh'] - df_vysl_010['t_n'].iloc[0]) / df_vysl_010['t_n'].iloc[0], 1) if df_vysl_010['t_n'].iloc[0] != 0 else 0
}])

# Přidání nového řádku do df_check
df_check = pd.concat([df_check, new_row], ignore_index=True)

df_check

,Úroveň,T Z AKT LHP,T Z AKT CHK,Rozdíl Z AKT,% Z AKT,T CVT LHP,T CVT CHK,Rozdíl CVT,% CVT,T N LHP,T N CHK,Rozdíl N,% N
0,LHP,1645871.3,1645871.3,0.0,0.0,523440.8,523440.8,0.0,0.0,5489137.3,5489137.3,0.0,0.0
1,VS,1645871.3,1645871.3,0.0,0.0,523440.8,523440.8,0.0,0.0,5489137.3,5489137.3,0.0,0.0
2,HOS,1645871.3,1658460.8,12589.5,0.8,523440.8,526242.5,2801.7,0.5,5489137.3,5519755.0,30617.7,0.6


In [21]:
del df_vysl_050_hos
del new_row
del sumar

## 50 LS

In [23]:
sql_query = f"""
 SELECT p.kod, p.nazev,
    round(pc.y_n, 1) AS y_n,
    round(pc.t_n, 0) AS pocet_kh,
    pc.y_z_akt AS y_z_akt,
    pc.t_z_akt AS zasoba,
    pc.y_cbp,
    pc.t_cbp,
    pc.y_cvt,
    pc.t_cvt,
	  round(p.plocha_ps, 2) plocha_ps
   FROM dsuhul.pobl_typy pt,
    dsuhul.pobl p,
    dsuhul.pobl_cz01 pc
  WHERE pt.uid_lhp = '{uid['lhp']}'::uuid AND pt.kod::text = 'LS'::text AND p.uid_tpobl = pt.uid AND pc.uid_pobl = p.uid AND pc.dr_zkr::text = ''::text
  ORDER BY p.kod
"""
df_vysl_050_ls = pd.read_sql(sql_query, engine)
if vystup_csv:
    df_vysl_050_ls.to_csv(data_dir + '/50_ls_sumar.csv', decimal=',', sep=';', index=False)
if vystup_excel:
    df_vysl_050_ls.to_excel(data_dir + '/50_ls_sumar.xlsx', index=False)
df_vysl_050_ls

,kod,nazev,y_n,pocet_kh,y_z_akt,zasoba,y_cbp,t_cbp,y_cvt,t_cvt,plocha_ps
0,10,Vranov,780.5,1944792.0,206.1,513680.0,8.1,20096.7,61.9,154366.9,2493.00
1,11,Habrůvka,769.0,2108508.0,270.0,740376.4,10.2,27916.2,87.5,239941.7,2743.78
2,13,Bílovice,812.2,1433125.0,221.9,391557.5,9.3,16357.0,73.3,129290.7,1765.33


In [24]:
# kontrola
sumar = df_vysl_050_ls[['zasoba', 't_cvt', 'pocet_kh']].sum()

# Vytvoření nového řádku pro df_check
new_row = pd.DataFrame([{
    'Úroveň': 'LS',
    'T Z AKT LHP': df_vysl_010['t_z_akt'].iloc[0],
    'T Z AKT CHK': sumar['zasoba'],
    'Rozdíl Z AKT': sumar['zasoba'] - df_vysl_010['t_z_akt'].iloc[0],
    '% Z AKT': round(100 * (sumar['zasoba'] - df_vysl_010['t_z_akt'].iloc[0]) / df_vysl_010['t_z_akt'].iloc[0],1) if df_vysl_010['t_z_akt'].iloc[0] != 0 else 0,
    'T CVT LHP': df_vysl_010['t_cvt'].iloc[0],
    'T CVT CHK': sumar['t_cvt'],
    'Rozdíl CVT': sumar['t_cvt'] - df_vysl_010['t_cvt'].iloc[0],
    '% CVT': round(100 * (sumar['t_cvt'] - df_vysl_010['t_cvt'].iloc[0]) / df_vysl_010['t_cvt'].iloc[0], 1) if df_vysl_010['t_cvt'].iloc[0] != 0 else 0,
    'T N LHP': df_vysl_010['t_n'].iloc[0],
    'T N CHK': sumar['pocet_kh'],
    'Rozdíl N': sumar['pocet_kh'] - df_vysl_010['t_n'].iloc[0],
    '% N': round(100 * (sumar['pocet_kh'] - df_vysl_010['t_n'].iloc[0]) / df_vysl_010['t_n'].iloc[0], 1) if df_vysl_010['t_n'].iloc[0] != 0 else 0
}])

# Přidání nového řádku do df_check
df_check = pd.concat([df_check, new_row], ignore_index=True)

df_check

,Úroveň,T Z AKT LHP,T Z AKT CHK,Rozdíl Z AKT,% Z AKT,T CVT LHP,T CVT CHK,Rozdíl CVT,% CVT,T N LHP,T N CHK,Rozdíl N,% N
0,LHP,1645871.3,1645871.3,0.0,0.0,523440.8,523440.8,0.0,0.0,5489137.3,5489137.3,0.0,0.0
1,VS,1645871.3,1645871.3,0.0,0.0,523440.8,523440.8,0.0,0.0,5489137.3,5489137.3,0.0,0.0
2,HOS,1645871.3,1658460.8,12589.5,0.8,523440.8,526242.5,2801.7,0.5,5489137.3,5519755.0,30617.7,0.6
3,LS,1645871.3,1645613.9,-257.4,-0.0,523440.8,523599.3,158.5,0.0,5489137.3,5486425.0,-2712.3,-0.0


In [25]:
del df_vysl_050_ls
del new_row
del sumar

## 50 LU

In [26]:
sql_query = f"""
 SELECT p.kod, p.nazev,
    round(pc.y_n, 1) AS y_n,
    round(pc.t_n, 0) AS pocet_kh,
    pc.y_z_akt AS y_z_akt,
    pc.t_z_akt AS zasoba,
    pc.y_cbp,
    pc.t_cbp,
    pc.y_cvt,
    pc.t_cvt,
	  round(p.plocha_ps, 2) plocha_ps
   FROM dsuhul.pobl_typy pt,
    dsuhul.pobl p,
    dsuhul.pobl_cz01 pc
  WHERE pt.uid_lhp = '{uid['lhp']}'::uuid AND pt.kod::text = 'LU'::text AND p.uid_tpobl = pt.uid AND pc.uid_pobl = p.uid AND pc.dr_zkr::text = ''::text
  ORDER BY p.kod
"""
df_vysl_050_lu = pd.read_sql(sql_query, engine)
if vystup_csv:
    df_vysl_050_lu.to_csv(data_dir + '/50_lu_sumar.csv', decimal=',', sep=';', index=False)
if vystup_excel:
    df_vysl_050_lu.to_excel(data_dir + '/50_lu_sumar.xlsx', index=False)
df_vysl_050_lu

,kod,nazev,y_n,pocet_kh,y_z_akt,zasoba,y_cbp,t_cbp,y_cvt,t_cvt,plocha_ps
0,10-01,Vranov Baba,801.4,650025.0,227.8,184735.1,10.6,8557.5,83.6,67804.1,811.29
1,10-02,Vranov Útěchov,754.1,554721.0,177.9,130866.4,9.4,6903.1,65.6,48235.5,735.84
2,10-03,Vranov Soběšice,781.2,738368.0,208.0,196546.8,4.9,4605.7,40.0,37827.5,945.86
3,11-01,Habrůvka Josefov,675.5,204895.0,277.4,84123.9,10.0,3032.6,87.0,26399.1,303.91
4,11-02,Habrůvka Rudice,643.4,385819.0,313.9,188212.8,10.2,6129.5,95.8,57462.2,569.56
5,11-03,Habrůvka Borky,832.4,541751.0,253.3,164826.8,9.0,5835.7,74.5,48495.2,651.50
6,11-04,Habrůvka Dřínová,864.1,639194.0,259.4,191877.6,10.9,8029.4,91.1,67388.1,739.78
7,11-05,Habrůvka Rakovec,757.8,362856.0,250.5,119969.1,10.9,5236.1,90.3,43241.6,479.02
8,13-01,Bílovice Vypálenky,902.1,517797.0,226.6,130044.7,8.3,4791.4,65.4,37546.7,574.57
9,13-02,Bílovice Hády,854.4,523294.0,193.5,118506.7,8.7,5332.0,66.2,40563.8,612.58


In [27]:
# kontrola
sumar = df_vysl_050_lu[['zasoba', 't_cvt', 'pocet_kh']].sum()

# Vytvoření nového řádku pro df_check
new_row = pd.DataFrame([{
    'Úroveň': 'LU',
    'T Z AKT LHP': df_vysl_010['t_z_akt'].iloc[0],
    'T Z AKT CHK': sumar['zasoba'],
    'Rozdíl Z AKT': sumar['zasoba'] - df_vysl_010['t_z_akt'].iloc[0],
    '% Z AKT': round(100 * (sumar['zasoba'] - df_vysl_010['t_z_akt'].iloc[0]) / df_vysl_010['t_z_akt'].iloc[0],1) if df_vysl_010['t_z_akt'].iloc[0] != 0 else 0,
    'T CVT LHP': df_vysl_010['t_cvt'].iloc[0],
    'T CVT CHK': sumar['t_cvt'],
    'Rozdíl CVT': sumar['t_cvt'] - df_vysl_010['t_cvt'].iloc[0],
    '% CVT': round(100 * (sumar['t_cvt'] - df_vysl_010['t_cvt'].iloc[0]) / df_vysl_010['t_cvt'].iloc[0], 1) if df_vysl_010['t_cvt'].iloc[0] != 0 else 0,
    'T N LHP': df_vysl_010['t_n'].iloc[0],
    'T N CHK': sumar['pocet_kh'],
    'Rozdíl N': sumar['pocet_kh'] - df_vysl_010['t_n'].iloc[0],
    '% N': round(100 * (sumar['pocet_kh'] - df_vysl_010['t_n'].iloc[0]) / df_vysl_010['t_n'].iloc[0], 1) if df_vysl_010['t_n'].iloc[0] != 0 else 0
}])

# Přidání nového řádku do df_check
df_check = pd.concat([df_check, new_row], ignore_index=True)

df_check

,Úroveň,T Z AKT LHP,T Z AKT CHK,Rozdíl Z AKT,% Z AKT,T CVT LHP,T CVT CHK,Rozdíl CVT,% CVT,T N LHP,T N CHK,Rozdíl N,% N
0,LHP,1645871.3,1645871.3,0.0,0.0,523440.8,523440.8,0.0,0.0,5489137.3,5489137.3,0.0,0.0
1,VS,1645871.3,1645871.3,0.0,0.0,523440.8,523440.8,0.0,0.0,5489137.3,5489137.3,0.0,0.0
2,HOS,1645871.3,1658460.8,12589.5,0.8,523440.8,526242.5,2801.7,0.5,5489137.3,5519755.0,30617.7,0.6
3,LS,1645871.3,1645613.9,-257.4,-0.0,523440.8,523599.3,158.5,0.0,5489137.3,5486425.0,-2712.3,-0.0
4,LU,1645871.3,1652131.6,6260.3,0.4,523440.8,525703.9,2263.1,0.4,5489137.3,5515713.0,26575.7,0.5


In [28]:
del df_vysl_050_lu
del new_row
del sumar

## 51 HOS VS

In [29]:
sql_query = f"""
   SELECT p.kod, p.nazev,
    vs.stratum,
    round(vsc.y_n, 1) AS y_n,
    round(vsc.t_n, 0) AS pocet_kh,
    vsc.y_z_akt AS y_z_akt,
    vsc.t_z_akt AS zasoba,
    vsc.t_cbp AS cbp,
	  (select round(sum(vp.plocha_ps), 2) from dsuhul.vs_pobl vp where vp.uid_pobl = p.uid and vp.uid_vs_slouc = vs.uid) plocha_ps
  FROM dsuhul.pobl_typy pt,
    dsuhul.pobl p,
    dsuhul.vs_pobl_cz01 vsc,
    dsuhul.vs vs,
	dsuhul.vs_pobl vsp
  WHERE pt.uid_lhp = '{uid['lhp']}'::uuid AND pt.kod::text = 'HOS'::text AND p.uid_tpobl = pt.uid AND vsc.uid_pobl = p.uid AND vs.uid = vsc.uid_vs AND vsc.dr_zkr::text = ''::text
    and vsp.uid_pobl = p.uid and vsp.uid_vs = vs.uid
  ORDER BY p.kod, vs.stratum
"""
df_vysl_051 = pd.read_sql(sql_query, engine)
if vystup_csv:
    df_vysl_051.to_csv(data_dir + '/51_hos_vs_sumar.csv', decimal=',', sep=';', index=False)
if vystup_excel:
    df_vysl_051.to_excel(data_dir + '/51_hos_vs_sumar.xlsx', index=False)
df_vysl_051

,kod,nazev,stratum,y_n,pocet_kh,y_z_akt,zasoba,cbp,plocha_ps
0,01,Účelové hospodářství v modelu Dauerwald nižšíc...,14,679.4,10204.0,259.864,3903.2,159.7,15.02
1,01,Účelové hospodářství v modelu Dauerwald nižšíc...,4,787.7,508662.0,215.395,139095.6,3176.5,645.77
2,01,Účelové hospodářství v modelu Dauerwald nižšíc...,5,848.8,224100.0,202.311,53414.2,2239.2,264.02
3,02,Účelové hospodářství v modelu Dauerwald středn...,10,827.9,108037.0,202.179,26384.4,632.9,130.50
4,02,Účelové hospodářství v modelu Dauerwald středn...,14,358.4,2978.0,285.664,2373.9,69.0,8.31
5,02,Účelové hospodářství v modelu Dauerwald středn...,6,761.3,251573.0,312.895,103402.4,3255.1,330.47
6,02,Účelové hospodářství v modelu Dauerwald středn...,7,806.1,3006295.0,235.604,878640.4,37938.3,3729.31
7,03,Účelové hospodářství v modelu Dauerwald středn...,6,928.0,216467.0,198.555,46316.9,1590.4,233.27
8,03,Účelové hospodářství v modelu Dauerwald středn...,7,610.9,119060.0,213.794,41664.2,1911.4,194.88
9,04,Účelové hospodářství v modelu výběrný les,8,524.8,33779.0,418.944,26963.2,541.4,64.36


In [30]:
del df_vysl_051

## 52 HOS DR

In [31]:
sql_query = f"""
 SELECT p.kod,
    pc.dr_zkr,
    sum(pc.t_n) AS pocet_kh,
    sum(pc.t_z_akt) AS zasoba,
    round(100::numeric * sum(pc.t_z_akt) / max(pcsum.t_z_akt), 1) AS podil_zasoby
   FROM dsuhul.pobl_typy pt,
    dsuhul.pobl p,
    dsuhul.pobl_cz01 pc,
    dsuhul.pobl_cz01 pcsum
  WHERE pt.uid_lhp = '{uid['lhp']}'::uuid AND pt.kod::text = 'HOS'::text AND p.uid_tpobl = pt.uid AND pc.uid_pobl = p.uid AND pc.dr_zkr::text <> ''::text 
      AND pcsum.uid_pobl = pc.uid_pobl AND pcsum.dr_zkr::text = ''::text AND pcsum.tlt = 0::numeric
  GROUP BY p.kod, pc.dr_zkr
  ORDER BY p.kod, pc.dr_zkr
"""
df_vysl_052 = pd.read_sql(sql_query, engine)
if vystup_csv:
    df_vysl_052.to_csv(data_dir + '/52_hos_dr_3d.csv', decimal=',', sep=';', index=False)
if vystup_excel:
    df_vysl_052.to_excel(data_dir + '/52_hos_dr_3d.xlsx', index=False)
df_vysl_052

,kod,dr_zkr,pocet_kh,zasoba,podil_zasoby
0,01,AK,767.6,270.3,0.1
1,01,BB,5891.2,966.8,0.5
2,01,BK,63442.7,15217.3,7.7
3,01,BO,94513.8,48576.2,24.7
4,01,BR,3383.0,1167.5,0.6
...,...,...,...,...,...
193,11,KL,544.8,374.4,1.1
194,11,LP,3663.1,1349.9,4.0
195,11,MD,74.1,81.0,0.2
196,11,OR,124.9,201.4,0.6


In [32]:
del df_vysl_052

## 53 HOS TLT

In [33]:
sql_query = f"""
 SELECT p.kod,
    pc.tlt,
    concat(min(ktlt.min_d13)::character varying(10), ' - ', max(ktlt.max_d13)::character varying(10)) AS interval_tlt,
    sum(pc.t_n) AS pocet_kh,
    sum(pc.t_z_akt) AS zasoba,
    round(100::numeric * sum(pc.t_z_akt) / max(pcsum.t_z_akt), 1) AS podil_zasoby
   FROM dsuhul.pobl_typy pt,
    dsuhul.pobl p,
    dsuhul.pobl_cz01 pc,
    dsuhul.pobl_cz01 pcsum,
    happ.lhp lhp,
    dsuhul.list_ktlt_types lktt,
    dsuhul.ktlt ktlt
  WHERE pt.uid_lhp = '{uid['lhp']}'::uuid AND pt.kod::text = 'HOS'::text AND p.uid_tpobl = pt.uid AND pc.uid_pobl = p.uid AND pc.dr_zkr::text <> ''::text AND pc.tlt > 0::numeric 
      AND pcsum.uid_pobl = pc.uid_pobl AND pcsum.dr_zkr::text = ''::text AND lhp.uid = pt.uid_lhp AND lktt.code::text = lhp.ktlt::text AND ktlt.uid_firm = lhp.uid_firm AND ktlt.uid_lktl = lktt.uid AND ktlt.trida = pc.tlt
  GROUP BY p.kod, pc.tlt
  ORDER BY p.kod, pc.tlt
"""
df_vysl_053 = pd.read_sql(sql_query, engine)
if vystup_csv:
    df_vysl_053.to_csv(data_dir + '/53_hos_tlt_3e.csv', decimal=',', sep=';', index=False)
if vystup_excel:
    df_vysl_053.to_excel(data_dir + '/53_hos_tlt_3e.xlsx', index=False)
df_vysl_053

,kod,tlt,interval_tlt,pocet_kh,zasoba,podil_zasoby
0,01,6.0,4.1 - 8.0,97160.9,887.3,0.5
1,01,10.0,8.1 - 12.0,212916.9,5740.1,2.9
2,01,14.0,12.1 - 16.0,123525.6,9714.1,4.9
3,01,18.0,16.1 - 20.0,83022.6,13971.0,7.1
4,01,22.0,20.1 - 24.0,58739.5,16412.1,8.4
...,...,...,...,...,...,...
189,11,70.0,68.1 - 72.0,156.4,857.3,2.5
190,11,74.0,72.1 - 76.0,74.1,588.2,1.7
191,11,78.0,76.1 - 80.0,74.1,599.4,1.8
192,11,82.0,80.1 - 84.0,74.9,513.5,1.5


In [34]:
del df_vysl_053

In [35]:
# zápis kontroly
if vystup_csv:
    df_check.to_csv(data_dir + '/90_kontrola.csv', decimal=',', sep=';', index=False)
if vystup_excel:
    df_check.to_excel(data_dir + '/90_kontrola.xlsx', index=False)
df_check

,Úroveň,T Z AKT LHP,T Z AKT CHK,Rozdíl Z AKT,% Z AKT,T CVT LHP,T CVT CHK,Rozdíl CVT,% CVT,T N LHP,T N CHK,Rozdíl N,% N
0,LHP,1645871.3,1645871.3,0.0,0.0,523440.8,523440.8,0.0,0.0,5489137.3,5489137.3,0.0,0.0
1,VS,1645871.3,1645871.3,0.0,0.0,523440.8,523440.8,0.0,0.0,5489137.3,5489137.3,0.0,0.0
2,HOS,1645871.3,1658460.8,12589.5,0.8,523440.8,526242.5,2801.7,0.5,5489137.3,5519755.0,30617.7,0.6
3,LS,1645871.3,1645613.9,-257.4,-0.0,523440.8,523599.3,158.5,0.0,5489137.3,5486425.0,-2712.3,-0.0
4,LU,1645871.3,1652131.6,6260.3,0.4,523440.8,525703.9,2263.1,0.4,5489137.3,5515713.0,26575.7,0.5


In [36]:
del df_check

In [37]:
sql_query = f"""
	SELECT
		pt.kod typ
		, p.kod kod
		, vs.stratum
		, vp.m_x
		, vp.plocha_ps
	FROM
		dsuhul.pobl_typy pt
		, dsuhul.pobl p
		, dsuhul.vs_pobl vp
		, dsuhul.vs vs
	WHERE
		pt.uid_firm = '{uid['firm']}'
		and p.uid_tpobl = pt.uid
		and vp.uid_pobl = p.uid
		and vs.uid = vp.uid_vs
	ORDER by pt.kod, p.kod, vs.stratum
"""
df_vysl_080 = pd.read_sql(sql_query, engine)
if vystup_csv:
    df_vysl_080.to_csv(data_dir + '/80_pobl_vs.csv', decimal=',', sep=';', index=False)
if vystup_excel:
    df_vysl_080.to_excel(data_dir + '/80_pobl_vs.xlsx', index=False)
df_vysl_080

,typ,kod,stratum,m_x,plocha_ps
0,HOS,01,14,5,14.67
1,HOS,01,2,1,0.35
2,HOS,01,4,135,645.77
3,HOS,01,5,62,264.02
4,HOS,02,10,16,130.50
...,...,...,...,...,...
89,LU,13-03,11,39,166.89
90,LU,13-03,12,34,139.48
91,LU,13-03,14,1,6.99
92,LU,13-03,7,25,211.43


In [38]:
del df_vysl_080

In [39]:
sql_query = f"""
	SELECT
		pt.kod typ
		, p.kod kod
		, vs.stratum vs_sloucene
		, (select string_agg(vss.stratum, ',' order by vss.stratum) 
			from dsuhul.vs_pobl vps, dsuhul.vs vss 
			where vps.uid_pobl = vp.uid_pobl and vps.uid_vs_slouc = vp.uid_vs_slouc and vss.uid = vps.uid_vs) strata
		, (select sum(vpm.m_x) from dsuhul.vs_pobl vpm where vpm.uid_pobl = vp.uid_pobl and vpm.uid_vs_slouc = vp.uid_vs_slouc) m_x
		, (select sum(vpp.plocha_ps) from dsuhul.vs_pobl vpp where vpp.uid_pobl = vp.uid_pobl and vpp.uid_vs_slouc = vp.uid_vs_slouc) plocha_ps
	FROM
		dsuhul.pobl_typy pt
		, dsuhul.pobl p
		, dsuhul.vs_pobl vp
		, dsuhul.vs vs
	WHERE
		pt.uid_firm = '{uid['firm']}'
		and p.uid_tpobl = pt.uid
		and vp.uid_pobl = p.uid and vp.uid_vs_slouc = vp.uid_vs
		and vs.uid = vp.uid_vs_slouc
	ORDER by pt.kod, p.kod, vs.stratum
"""
df_vysl_081 = pd.read_sql(sql_query, engine)
if vystup_csv:
    df_vysl_081.to_csv(data_dir + '/81_pobl_vs_slouceni.csv', decimal=',', sep=';', index=False)
if vystup_excel:
    df_vysl_081.to_excel(data_dir + '/81_pobl_vs_slouceni.xlsx', index=False)
df_vysl_081

,typ,kod,vs_sloucene,strata,m_x,plocha_ps
0,HOS,01,14,"14,2",6,15.02
1,HOS,01,4,4,135,645.77
2,HOS,01,5,5,62,264.02
3,HOS,02,10,10,16,130.50
4,HOS,02,14,"11,12,14,4,9",3,8.31
...,...,...,...,...,...,...
75,LU,13-02,7,7,16,134.65
76,LU,13-03,11,11,39,166.89
77,LU,13-03,12,12,34,139.48
78,LU,13-03,7,7,25,211.43


In [40]:
del df_vysl_081
del df_vysl_010